In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'fmnist'
params['dataset'] = 'sent140'
#params['model'] = 'mlp'
params['model'] = 'stacked_gru'
params['optimizer'] = 'fedprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'celeba':
  params['model_params'] = (2,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'vehicle':
  params['model_params'] = (2,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 2
if params['dataset'] == 'fmnist':
  params['model_params'] = (10,)
  params['learning_rate'] = 0.05
  params['mu'] = 1
  params['num_group'] = 3

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['MADC'] = False                       # If false, EDC will be used
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['recluster_epoch'] = None
  params['client_temp'] = None
  params['allow_empty'] = True
  params['agg_lr'] = 0.0 ########## Group Aggregataion Rate
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = False
  params['fesem'] = False
  
  """
  We immplement IFCA and FeSEM base on grouprox,
  Some configurations should be flush for above two CFL frameworks
  """
  if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False ### Do not modify it
    params['RAC'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0.0 # IFCA and FeSEM don't use this parameter
    
"""
Common parameters
"""
params['num_rounds'] = 800
params['num_epochs'] = 10
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  if params['ifca'] == False and params['fesem'] == False:
    if params['MADC'] == True:
      info2 = info2 + '-MADC'
    else:
      info2 = info2 + '-EDC'
    if params['recluster_epoch']:
        info2 = info2 + '-RE{:2d}'.format(params['recluster_epoch'])
    if params['client_temp']:
        info2 = info2 + '-TEMP{:2d}'.format(params['client_temp'])
        
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = info2 +  '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

fedprox-sent140-stacked_gru.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Federated prox to Train
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/lab/anaconda3/envs/flex/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:524: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.GRUCell` is deprecated and will be removed "
/home/lab/anaconda3/envs/flex/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/lab/anaconda3/envs/flex/lib/python3.9/site-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/lab/anaconda3/envs/flex/lib/python3.9/site-package

Instructions for updating:
Use `tf.cast` instead.


2021-11-01 12:18:45.919088: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-01 12:18:46.321409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22314 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

In [ ]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

In [ ]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

In [4]:
server.train()

Training with 20 workers ---
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


2021-11-01 12:18:53.806282: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


At round 0 accuracy: 0.613036303630363
At round 0 training accuracy: 0.5512321660181583
At round 0 training loss: 0.695581448449235
At round 0 Discrepancy: 0
gradient difference: 0.8593350097529829


At round 1 accuracy: 0.6100895803866101
At round 1 training accuracy: 0.5040816326530613
At round 1 training loss: 0.7067288722310747
At round 1 Discrepancy: 1.1431233988661869
gradient difference: 1.1704158667374747


At round 2 accuracy: 0.5874587458745875
At round 2 training accuracy: 0.6099865047233468
At round 2 training loss: 0.6727080148884803
At round 2 Discrepancy: 1.6419283449711277
gradient difference: 1.8605427488238633


...

At round 796 accuracy: 0.7056812824139557
At round 796 training accuracy: 0.8176100628930818
At round 796 training loss: 0.4578259189680523
At round 796 Discrepancy: 2.3692194458873375
gradient difference: 5.308729483574377


At round 797 accuracy: 0.7184111268269684
At round 797 training accuracy: 0.8419071518193224
At round 797 training loss: 0.39540881895398255
At round 797 Discrepancy: 1.6383902364889096
gradient difference: 5.105404723722034


At round 798 accuracy: 0.7234794908062234
At round 798 training accuracy: 0.8579088471849866
At round 798 training loss: 0.389942728412455
At round 798 Discrepancy: 1.8715065357051142
gradient difference: 5.572896788487846


At round 799 accuracy: 0.7243045733144743
At round 799 training accuracy: 0.8438356164383561
At round 799 training loss: 0.4708479014161515
At round 799 Discrepancy: 1.5258426518640111
gradient difference: 6.362471314891438


At round 800 accuracy: 0.7141678453559641
At round 800 training accuracy: 0.8391591070930988


In [ ]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""

In [ ]:
a = np.arange(32).reshape(2,4,4)
mask = np.zeros(shape=(4,4))
np.fill_diagonal(mask, 1)
#for i in range(a.shape[0]):
#    mask[i,:,i] =1
#x = np.ma.array([a, mask=[0] + [1,0]*4)
b = np.repeat(mask[np.newaxis,:,:], 2, axis=0)
for i in range(b.shape[0]):
    b[i,:,i]=1
    #b[i,i,:]=1
print(a)
x=np.ma.array(a,mask=b)
print(x)
print(np.sum(x,axis=-1))